In [ ]:
using BasisFunctions, FrameFun, Plots, DomainSets, LinearAlgebra

# 1. Introduction 

Using frames, the behaviour of the coefficients is not predictable. This is easily seen comparing a Chebyshev extension approximation with a Chebyshev series approximation.

In [ ]:
CE = FrameFun.ExtensionFramePlatform(platform(ChebyshevT(1,-2,2)), -1..1)
CS = ChebyshevPlatform()

We approximate $f(x)=e^x$ on `-1..1`,

In [ ]:
f = exp;

Using two approximation sizes

In [ ]:
N1 = 20; N2 = 40;

and see that,  for a basis,  coefficient size goes down to machine precision, for a frame, coefficients go down; but they do not reach a platform. 

In [ ]:
plot(abs.(coefficients(Fun(f, CS, N1))).+eps(),yscale=:log10,layout=2,ylims=[1e-16,10],label="Small",title="Series",size=(900,300),linestyle=:dash,marker=:dot)
scatter!(abs.(coefficients(Fun(f, CS, N2))).+eps(),label="Large",legend=:topright)
plot!(abs.(coefficients(Fun(f, CE, N1))).+eps(),yscale=:log10,subplot=2,ylims=[1e-16,10],label="Small",title="Extension",linestyle=:dash,marker=:dot)
scatter!(abs.(coefficients(Fun(f, CE, N2))).+eps(),subplot=2,label="Large",legend=:bottomleft)

# 2. Smoothing

In this section we approximate the exponential function with a Fourier extension

In [ ]:
f = exp
P = FourierExtensionPlatform(0..0.5)

Smoothing penalizes the high frequencies. In stead of solving `Ax=b` in a least squares sense, we solve $AD^{-1}y=b$ and $x=D^{-1}y$. Such that $\|Dx\|$ is minimized while solving `Ax=b`. D is here a diagonal operator where diagonal entries depend on the frequency. The dependency of the weights on the frequency is given by

In [ ]:
function scaling(dict, idx)
    l = abs(value(native_index(dict, idx)))
    1e-4+l+l^2
end;

We solve the approximation problem two times. One in the ordinary least squares sense and one penalizing the high frequencies. The smoothed approximation has a nicer extension, but its coefficients do not go down fast enough to make  predictions on the approximation error. 

In [ ]:
F = Fun(f, P, 61; solverstyle=AZStyle())
FS = Fun(f, P, 61; solverstyle=AZSmoothStyle(), scaling=scaling)

In [ ]:
scatter(;layout=(1,3),size=(900,300))
plot!(F,subplot=1,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,subplot=1,label="smooth",plot_ext=true)
plot!(F,f,subplot=2,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,f,subplot=2,label="smooth",plot_ext=true)
scatter!(abs.(coefficients(F)), yscale=:log10,subplot=3,legend=false)
scatter!(abs.(coefficients(FS)), yscale=:log10,subplot=3)

Again, using more degrees of freedom does not lead to a smallest coefficient size that is smaller $\mathcal O(10^{-9})$

In [ ]:
F = Fun(f, P, 101; solverstyle=AZStyle())
FS = Fun(f, P, 101; solverstyle=AZSmoothStyle(),scaling=scaling)

In [ ]:
scatter(;layout=(1,3),size=(900,300))
plot!(F,subplot=1,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,subplot=1,label="smooth",plot_ext=true)
plot!(F,f,subplot=2,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,f,subplot=2,label="smooth",plot_ext=true)
scatter!(abs.(coefficients(F)), yscale=:log10,subplot=3,legend=false)
scatter!(abs.(coefficients(FS)), yscale=:log10,subplot=3)

# 3. Adaptive approximation

We study the behaviour of adaptive frame approximation using platforms. 

## 3.1. Function with singularity just outside the interval of interest

We choose to approximate a function with a singularity to the right of the interval of interest (`0.0..0.5`). 

In [ ]:
f = x->.1/(x-.6);

The function is approximated using a Fourier Extension platform. 

In [ ]:
P = FourierExtensionPlatform(0.0..0.5);

We use different stopping criteria and evaluate the properties of the resulting approximations. 

In [ ]:
ResidualStyle(), FNAStyle()

### a) Using the ResidualStyle

The `ResidualStyle()` `criterion` measures the norm of the residual of the system Ax=B; and compares it with a tolerance `tol`. 

In [ ]:
FRES = Fun(f, P, criterion = ResidualStyle(), threshold=1e-10,verbose=true)

### b) Using the FNAStyle

The `FNAStyle()` `criterion` checks two subcriteria. Firstly, it measures the norm of the coefficients. The $\ell^2$-norm of the coefficients should be smaller than `FNAeta`*$\|f\|$. Secondly, we measure the residual of the system Ax=B is measured; and compare it with a tolerance `tol` (as in `ResidualStyle()`).  

In [ ]:
FFNA = Fun(f, P, criterion = FNAStyle(), FNAeta=5, threshold=1e-10,verbose=true)

### c) Using the residual style, but with small coefficients 

As before, the `ResidualStyle()` `criterion` measures the norm of the residual of the system Ax=B; and compares it with a tolerance `tol`. Moreover we restrict our solution space to one with small coefficients using the `smallcoefficients=true` option. With the `smallcoefficients_rtol` we eliminate coefficient components of the solution larger than `smallcoefficients_rtol`$\|f\|$. (Note, the `smallcoefficients` option works using the `AZSolver` combined with the `RandomizedSvdSolver` only.)

In [ ]:
FRES_cap = Fun(f, P, criterion = ResidualStyle(), threshold=1e-10,verbose=true, smallcoefficients=true, smallcoefficients_rtol = 5)

### Comparison

`FNAStyle` is more restrictive than `ResidualStyle`; therefore, it is no suprise that `FNAStyle` gives an approximation with more degrees of freedom. A similar reasoning holds comparing `ResidualStyle` with and without the `smallcoefficients` option.

In [ ]:
length(FRES), length(FFNA), length(FRES_cap)

At the upside, if coefficients are smaller, we obtain a more robust approximation. Here this is visible by the size of the extension of the approximation. The extension is a lot larger using `ResidualStyle`.  

In [ ]:
plot(FFNA; plot_ext=true,label="FNA",layout=2,size=(900,300))
plot!(FRES; plot_ext=true,label="RES",legend=:topleft)
plot!(FRES_cap; plot_ext=true,label="RES_cap",legend=:topleft)
plot!(FFNA,f; plot_ext=true,label="FNA",subplot=2)
plot!(FRES,f; plot_ext=true,label="RES",legend=:topleft,subplot=2)
plot!(FRES_cap,f; plot_ext=true,label="RES_cap",legend=:topleft,subplot=2)

Another motivation for using the coeffient norm as a measure for a good approximation in the `FNAStyle` is seen by comparing the coefficient sizes, approximation errors, and residuals of approximations with different approximation sizes. From <a href="https://arxiv.org/abs/1802.01950">FaNA II</a> is known that the coefficient size will converge to $\tfrac{\|f\|}{\sqrt A}$. Where $A$ is the lower frame bound of the Fourier extension frame. Nothing is known about its convergence speed; therefore, we allow for some constant `FNAeta`.

In [ ]:
f = x->.1/(x-.55);

In [ ]:
N = 10:10:400
plot(;yscale=:log10,ylims=[1e-16,1e12],layout=(2,2),size=(900,500))
for (i,threshold) in enumerate([1e-3,1e-6,1e-9,1e-12])
    F = [Fun(f, P, Ni; threshold=threshold) for Ni in N]
    coefsize = [norm(coefficients(Fi)) for Fi in F];
    residuals = [residual(f, Fi) for Fi in F];
    L2errors = [L2error(f, Fi;rtol=1e-10,atol=1e-10) for Fi in F];
    Fnorm = ones(length(N))*(0.24/sqrt(2));
    labels = (i != 2) ? ["","","",""] : ["Size","Residual","L2 error","||f||"]
    scatter!(N,coefsize;yscale=:log10,ylims=[1e-16,1e5],subplot=i,label=labels[1])
    scatter!(N,residuals,subplot=i,label=labels[2])
    scatter!(N,L2errors,subplot=i,label=labels[3])
    plot!(N,Fnorm,linestyle=:dash,subplot=i,label=labels[4])
    plot!(N,threshold*ones(length(N)),linestyle=:dash,c=:black,subplot=i,label="")
end

In [ ]:
plot!()

Below we create a similar plot as the one above, but bound the coefficients. 

In [ ]:
N = 10:10:400
plot(;yscale=:log10,ylims=[1e-16,1e12],layout=(2,2),size=(900,500))
for (i,threshold) in enumerate([1e-3,1e-6,1e-9,1e-12])
    F = [Fun(f, P, Ni; threshold=threshold,smallcoefficients=true,smallcoefficients_rtol=5) for Ni in N]
    coefsize = [norm(coefficients(Fi)) for Fi in F];
    residuals = [residual(f, Fi) for Fi in F];
    L2errors = [L2error(f, Fi;rtol=1e-10,atol=1e-10) for Fi in F];
    Fnorm = ones(length(N))*(0.24/sqrt(2));
    labels = (i != 2) ? ["","","",""] : ["Size","Residual","L2 error","||f||"]
    scatter!(N,coefsize;yscale=:log10,ylims=[1e-16,1e5],subplot=i,label=labels[1])
    scatter!(N,residuals,subplot=i,label=labels[2])
    scatter!(N,L2errors,subplot=i,label=labels[3])
    plot!(N,Fnorm,linestyle=:dash,subplot=i,label=labels[4])
    plot!(N,threshold*ones(length(N)),linestyle=:dash,c=:black,subplot=i,label="")
end

In [ ]:
plot!()

## 3.2. High oscillatory function

We approximate a high oscillatory on (`0.0..0.5`). 

In [ ]:
f = x->cos(200x^2);

The function is approximated using a Fourier Extension platform. 

In [ ]:
P = FourierExtensionPlatform(0.0..0.5);

We use different stopping criteria and evaluate the properties of the resulting approximations. 

In [ ]:
ResidualStyle(), FNAStyle()

### a) Using the ResidualStyle

The `ResidualStyle()` `criterion` measures the norm of the residual of the system Ax=B; and compares it with a tolerance `tol`. 

In [ ]:
FRES = Fun(f, P, criterion = ResidualStyle(), threshold=1e-10,verbose=true)

### b) Using the FNAStyle

The `FNAStyle()` `criterion` checks two subcriteria. Firstly, it measures the norm of the coefficients. The $\ell^2$-norm of the coefficients should be smaller than `FNAeta`*$\|f\|$. Secondly, we measure the residual of the system Ax=B is measured; and compare it with a tolerance `tol` (as in `ResidualStyle()`).  

In [ ]:
FFNA = Fun(f, P, criterion = FNAStyle(), FNAeta=5, threshold=1e-10,verbose=true)

### Comparison

Again, `FNAStyle` gives an approximation with more degrees of freedom. 

In [ ]:
length(FRES), length(FFNA) 

At the upside, coefficients are smaller. This gives a more robust approximation. Here this is visible by the size of the extension of the approximation. The extension is a lot larger using `ResidualStyle`.  

In [ ]:
plot(FFNA; plot_ext=true,label="FNA",layout=2,size=(900,300))
plot!(FRES; plot_ext=true,label="RES",legend=:topleft)
plot!(FFNA,f; plot_ext=true,label="FNA",subplot=2)
plot!(FRES,f; plot_ext=true,label="RES",legend=:topleft,subplot=2)

The highest frequency in the function is 50. Therefore, the function starts to converge when the number of degrees of freedom reach 50. As is seen in the figure below, coefficients increase to $\frac{1}{\epsilon}$ until convergence is  started.

In [ ]:
N = 10:10:150
plot(;yscale=:log10,ylims=[1e-16,1e12],layout=(2,2),size=(900,500))
for (i,threshold) in enumerate([1e-3,1e-6,1e-9,1e-12])
    F = [Fun(f, P, Ni; threshold=threshold) for Ni in N]
    coefsize = [norm(coefficients(Fi)) for Fi in F];
    residuals = [residual(f, Fi) for Fi in F];
    L2errors = [L2error(f, Fi;rtol=1e-5,atol=1e-5) for Fi in F];
    Fnorm = ones(length(N))*(0.24/sqrt(2));
    labels = (i != 4) ? ["","","","",""] : ["Size","Residual","L2 error","||f||","tolerance"]
    scatter!(N,coefsize;yscale=:log10,subplot=i,label=labels[1])
    scatter!(N,residuals,subplot=i,label=labels[2])
    scatter!(N,L2errors,subplot=i,label=labels[3])
    plot!(N,Fnorm,linestyle=:dash,subplot=i,label=labels[4])
    plot!(N,threshold*ones(length(N)),linestyle=:dash,c=:black,subplot=i,label=labels[5])
end

In [ ]:
plot!()

## 3.3. Function with singularity just outside the interval of interest and at the interval boundary

In [ ]:
f = x-> sqrt(sin(x)) + sin(20x) + .1/(x-.55);

In [ ]:
plot(f, LinRange(0,.5,100);legend=false, size=[400,200])

In [ ]:
using Calculus
struct SQRT <: Function 
end

(::SQRT)(x::T) where T<:Number = sqrt(x)

struct DIFFSQRT <: Function 
end
Calculus.derivative(::SQRT) = DIFFSQRT()
Calculus.derivative(::SQRT, x::T) where T<:Number = (DIFFSQRT())(x)
(::DIFFSQRT)(x::T) where T<:Number = one(T)/(2sqrt(x))

In [ ]:
WP = WeightedSumPlatform(P, SQRT(),x->1)

In [ ]:
FRES = Fun(f, WP, criterion = ResidualStyle(), threshold=1e-12,verbose=true,oversamplingfactor=5)

In [ ]:
plot(FRES, f; layout=2)
plot!(FRES; plot_ext=true, subplot=2)

In [ ]:
FFNA = Fun(f, WP, criterion = FNAStyle(), FNAeta=5, threshold=1e-12,verbose=true, maxlength=1<<14,oversamplingfactor=5)

### Comparison

In [ ]:
length(FRES), length(FFNA)

In [ ]:
plot(FFNA; plot_ext=true,label="FNA",layout=2,size=(900,300))
plot!(FRES; plot_ext=true,label="RES",legend=:topleft)
plot!(FFNA,f; plot_ext=true,label="FNA",subplot=2)
plot!(FRES,f; plot_ext=true,label="RES",legend=:topleft,subplot=2)

In [ ]:
f = x-> sqrt(sin(x)) + sin(20x) + .1/(x-.55);

In [ ]:
N = 10:10:400
plot(;yscale=:log10,ylims=[1e-16,1e12],layout=(2,2),size=(900,500))
for (i,threshold) in enumerate([1e-3,1e-6,1e-9,1e-12])
    F = [Fun(f, WP, (Ni,Ni); threshold=threshold) for Ni in N]
    coefsize = [norm(coefficients(Fi)) for Fi in F];
    residuals = [residual(f, Fi) for Fi in F];
    L2errors = [L2error(f, Fi;rtol=1e-10,atol=1e-10) for Fi in F];
    Fnorm = ones(length(N))*(0.0857645);
    labels = (i != 2) ? ["","","",""] : ["Size","Residual","L2 error","||f||"]
    scatter!(N,coefsize;yscale=:log10,ylims=[1e-16,1e5],subplot=i,label=labels[1])
    scatter!(N,residuals,subplot=i,label=labels[2])
    scatter!(N,L2errors,subplot=i,label=labels[3])
    plot!(N,Fnorm,linestyle=:dash,subplot=i,label=labels[4])
    plot!(N,threshold*ones(length(N)),linestyle=:dash,c=:black,subplot=i,label="")
end

In [ ]:
plot!()